Solve for the nodal displacements and the reaction force for the following structure:

![](./images/structure.png)

In [1]:
import numpy as np

In [2]:
# Let's first define some constants
# We'll use the mm-kg-sec unit system
# The mm-kg-sec unit system has force units of N and 
# pressure units of MPa

E = 206.8e3      # MPa

L1 = 100    # length of elements 1 and 2 in mm
L3 = 200    # length of element 3 in mm

A1 = 1000   # Areas of elements 1 and 2 in mm^2
A3 = 100    # Area of element 3 in mm^2

We know from the solution to HW1 that the global stiffness matrix has the following form:

![](./images/structure_matrix_equation.png)

where $F_1$ is the unknown reaction force $R$, $F_2$ is -10,000 N, $F_3$ is 0, and $F_4$ is 5,000 N

We'll define $k_1$ to be $\frac{EA_1}{L1}$ and $k_3$ to be $\frac{EA_3}{L3}$. Note that since the second element as the same properties as element 1, $k_2=k_1$.

In [3]:
# Define the element k values (k1 and k3)
k1 = (E*A1)/L1  # element 2 will be the same
k3 = (E*A3)/L3

In [4]:
# Define the global stiffness matrix as a numpy array
K = np.array([[k1, -k1, 0, 0],
              [-k1, 2*k1, -k1, 0],
              [0, -k1, k1+k3, -k3],
              [0, 0, -k3, k3]])
K

array([[ 2068000., -2068000.,        0.,        0.],
       [-2068000.,  4136000., -2068000.,        0.],
       [       0., -2068000.,  2171400.,  -103400.],
       [       0.,        0.,  -103400.,   103400.]])

In [5]:
# To solve this problem, we'll need to multiply both sides of the above 
# matrix equation by the inverse of the stiffness matrix
# Let's see if the stiffness matrix is invertable by calculating its rank
rank = np.linalg.matrix_rank(K)
rank

3

The rank of a square matrix is a count of the number of linearily indpendent rows. For a square matrix to be invertable, its rank must equal its number of rows. In this case the rank needs to be 4. A rank of less than 4 is expected since, without any displacement boundary condtions, the  stiffness matrix is not invertable. To make it invertable, we need to apply the displacement boundary condition. For this problem, the displacement boundary condition is $\delta_1=0$. To applying this boundary condition, we'll change the first row of the matrix equation to $\delta_1=0$. After apply this boundary condition, the K matrix will become invertable.


In [6]:
# Redefine K and define the force vector F with the displacement boundary condition
K_with_bc = np.array([[1, 0, 0, 0],
                      [-k1, 2*k1, -k1, 0],
                      [0, -k1, k1+k3, -k3],
                      [0, 0, -k3, k3]])

F_with_bc = np.array([0,
                      -10000,
                      0,
                      5000])

K_with_bc, F_with_bc

(array([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0680e+06,  4.1360e+06, -2.0680e+06,  0.0000e+00],
        [ 0.0000e+00, -2.0680e+06,  2.1714e+06, -1.0340e+05],
        [ 0.0000e+00,  0.0000e+00, -1.0340e+05,  1.0340e+05]]),
 array([     0, -10000,      0,   5000]))

In [7]:
# Calculate the rank of the updated stiffness matrix with boundary condition appplied
rank = np.linalg.matrix_rank(K_with_bc)
rank

4

In [8]:
# Invert this new K and muliply by the force vector to obtain the displacements
delta = np.linalg.inv(K_with_bc)@F_with_bc
delta  # these displacements will be in mm

array([ 1.72698058e-18, -2.41779497e-03,  1.73472348e-18,  4.83558994e-02])

Finally, we need to caclulate the reaction force due to the applied boundary condition. To do this, we use the original matrix equation before applying the displacement boundary condition to obtain to full foce fector

In [9]:
# Calculate the reaction force
reaction_force = K@delta
reaction_force # these forces will be in N

array([ 5.00000000e+03, -1.00000000e+04, -9.09494702e-13,  5.00000000e+03])